In [1]:
print('Dit duurt tussen een seconde en bijna 2 minuten. Wacht tot de output eindigt met "Done."\n\n')
%run ./__init__edwh__notebooks.ipynb

Dit duurt tussen een seconde en bijna 2 minuten. Wacht tot de output eindigt met "Done."


Python 3.9.7 | packaged by conda-forge | (default, Sep 29 2021, 19:20:46) 
[GCC 9.4.0]
 * postgresql://postgres@pgpool:5432/backend
Done.


# Sticker toevoegen
Om stickers toe te voegen is dit de "handmatige" procedure: 
##  Bijlage uploaden 
 * open https://web2py.[domain]/init/default/quick_edit/0da1c5e9-4572-4da6-af4f-14c277c90c97
 * voeg onderaan de pagina de `.svg` bijlage toe. 
 * wacht een paar seconden en ververs de pagina om te zien dat de bijlage geupload is. Deze zou nu onderaan de bijlagen moeten staan. 
 * open de svg in een nieuw tabblad, en kopieer de GID van de bijlage en plak deze hieronder in de variabele `ATTACHMENT_GID`

In [2]:
ATTACHMENT_GID = '2ff8a8b6-d7b5-48be-ac8e-22f8227df4dc'

## Tag informatie opgeven
Voor een nieuwe tag zijn een `name` en `description` vereist.  
Vul deze gegevens hieronder in. 

Elke tag wordt aangemaakt als child van stickers, en mag gemetatagged zijn met `Item`, maar is niet nodig. 


In [3]:
STICKER_NAME='testje3'
STICKER_DESCRIPTION='langere omschrijving3'

Nu gaan we uitzoeken of de tag uberhaupt al bestaat of niet. 

In [4]:
TStickers = Tag('Stickers')
try: 
    new_tag = Tag(STICKER_NAME)
except KeyError: 
    # dit is goed, want we willen dat hij dit niet kan vinden. 
    new_tag = Tag.new(STICKER_NAME, STICKER_DESCRIPTION, parents=[TStickers])
    db.commit() 
    Tag.refresh(Tag.db)
    TStickers = Tag('Stickers') 
else: 
    print(f'"{STICKER_NAME}" bestaat al als Tag')
    

"testje3" bestaat al als Tag


In [5]:
print(new_tag)

<Tag testje3 <Row {'id': 32615, 'platform': None, 'gid': '74177976-dffb-4c0c-87a2-2a9f8645c15a', 'name': 'testje3', 'slug': 'testje3', 'parents': ['67ba8cd8-b564-4cb4-b1bc-10364c5edf16'], 'children': [], 'description': 'langere omschrijving3', 'meta_tags': []}>>


In [6]:
%%sql 
select * from tag
-- where parents like '%|67ba8cd8-b564-4cb4-b1bc-10364c5edf16|%'
where slug like '%'||'{STICKER_NAME}'||'%'

 * postgresql://postgres@pgpool:5432/backend
1 rows affected.


id,platform,gid,name,slug,parents,description,meta_tags,children
32615,None,74177976-dffb-4c0c-87a2-2a9f8645c15a,testje3,testje3,|67ba8cd8-b564-4cb4-b1bc-10364c5edf16|,langere omschrijving3,||,||


In [7]:
TStickers.walk(_print)

 Stickers ['System']
    SlimFit onderwijs ['User generated tags', 'Item']
    Leerling2020 ['User generated tags', 'Item']
    NRO Kennisrotonde []
    Voortgezet Leren []
    TOM (Team Onderwijs op Maat) []
    Slimmerlerenmetict.nl -> PO Raad []
    LOF (Lerarenontwikkelfonds) []
    Leraren met Lef []
    Leraar24.nl []
    OnderwijsInc []
    leraar.nl []
    Kennisnet podcasts []
    DNA van de leraar []
    voordeklas.com []
    Schoolleiders voor de Toekomst []
    VO Raad []
    PACT voor Kindcentra []
    stichting leerKRACHT []
    vernieuwenderwijs []
    Facebook groepen []
    Reformatorisch Dagblad []
    Eenvandaag []
    test-sticker ['System']
    HUPP ['System']
    Lesopafstand.nl ['System']
    School aan Zet ['System']
    Leren met de Stad ['System']
    Meesterwerk Podcast ['System']
    Gemeente Leiden ['System']
    testje []
    testje2 []
    testje3 []


##  Koppel de tag aan de attachment

In [8]:
%%sql
select * from attachment 
where gid = :ATTACHMENT_GID

 * postgresql://postgres@pgpool:5432/backend
1 rows affected.


id,platform,gid,attachment,filename,purpose,ts_uploaded,owner_gid,b2_uri
3827,SvS,2ff8a8b6-d7b5-48be-ac8e-22f8227df4dc,attachment.attachment.ba84196b7767269e.32313630396365392d303639302d346532302d616265352d6236326462653065626533382e737667.svg,21609ce9-0690-4e20-abe5-b62dbe0ebe38.svg,attachment,2021-05-20 10:01:56,3c8884bd-4fb4-485c-bb57-4c1d9e712395,https://f003.backblazeb2.com/file/nl-meteddie-delen-attachments/2ff8a8b6-d7b5-48be-ac8e-22f8227df4dc.svg


In [9]:
db.rollback()

In [10]:
if db((db.sticker.attachment_gid==ATTACHMENT_GID)&(db.sticker.tag_gid==new_tag.gid)).count() == 0:
    print('sticker nog niet gekoppeld aan attachment')
    db.sticker.insert(tag_gid=new_tag.gid, attachment_gid=ATTACHMENT_GID)
    print('nu wel')
    db.commit()
else:
    print('sticker reeds gekoppeld aan attachment')

sticker reeds gekoppeld aan attachment


In [11]:
%%sql 
select sticker.*, tag.slug, attachment.filename
  from sticker
    left outer join tag on sticker.tag_gid = tag.gid 
    left outer join attachment on sticker.attachment_gid=attachment.gid 
 where attachment_gid = :ATTACHMENT_GID 

 * postgresql://postgres@pgpool:5432/backend
3 rows affected.


id,tag_gid,attachment_gid,slug,filename
29,21609ce9-0690-4e20-abe5-b62dbe0ebe38,2ff8a8b6-d7b5-48be-ac8e-22f8227df4dc,gemeente-leiden,21609ce9-0690-4e20-abe5-b62dbe0ebe38.svg
32,c5a80e94-bf3d-4672-a560-9f2c65f3f43d,2ff8a8b6-d7b5-48be-ac8e-22f8227df4dc,testje2,21609ce9-0690-4e20-abe5-b62dbe0ebe38.svg
35,74177976-dffb-4c0c-87a2-2a9f8645c15a,2ff8a8b6-d7b5-48be-ac8e-22f8227df4dc,testje3,21609ce9-0690-4e20-abe5-b62dbe0ebe38.svg
